In [55]:
import pandas as pd
import numpy as np
from collections import Counter

In [141]:
train_df = pd.read_csv('data/train_sent_emo.csv')
train_df.head(5)

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [142]:
print(df['Emotion'].describe(), '\n')

cntr = Counter(train_df['Emotion'])
for e in sorted(cntr.items(), key=lambda x: x[1], reverse=True):
    print(*e)

count        9989
unique          7
top       neutral
freq         4710
Name: Emotion, dtype: object 

neutral 4710
joy 1743
surprise 1205
anger 1109
sadness 683
disgust 271
fear 268


In [123]:
test_df = pd.read_csv('data/test_sent_emo.csv')
test_df.head(5)

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [124]:
# Determine zero rule baseline on test labels
total_labels = len(test_df['Emotion'])
c = Counter(test_df['Emotion'])
prediction = max(c, key=lambda k: c[k])
accuracy = c[prediction] / total_labels * 100
print(f'predict=: {prediction}. accuracy= {cntr[prediction]} / {total_labels} or {accuracy:.2f}%')

predict=: neutral. accuracy= 4710 / 2610 or 48.12%


In [143]:
def add_dia_utt(dataframe):
    # feature vectors are stored by 'dia_utt' so create that in the df
    cols = ['Dialogue_ID', 'Utterance_ID']
    dataframe['dia_utt'] = dataframe[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

add_dia_utt(train_df)
train_df['dia_utt']

0           0_0
1           0_1
2           0_2
3           0_3
4           0_4
         ...   
9984    1038_13
9985    1038_14
9986    1038_15
9987    1038_16
9988    1038_17
Name: dia_utt, Length: 9989, dtype: object

In [144]:
train_df.count()

Sr No.          9989
Utterance       9989
Speaker         9989
Emotion         9989
Sentiment       9989
Dialogue_ID     9989
Utterance_ID    9989
Season          9989
Episode         9989
StartTime       9989
EndTime         9989
dia_utt         9989
dtype: int64

__audio_embeddings_feature_selection_emotion.pkl__

__audio_embeddings_feature_selection_sentiment.pkl__ 

These consist of 1611 dimensional audio feature vectors of each utterance trained for emotion/sentiment classification. These features are originally extracted from openSMILE and then followed by L2-based feature selection using SVM. It is a list comprising of 3 dictionaries for train, val and the test set with each dictionary indexed in the format dia_utt, where dia is the dialogue id and utt is the utterance id. For eg. train_audio_emb['0_0'].shape = (1611, )

In [43]:
import pickle

fpath = 'data/audio_embeddings_feature_selection_emotion.pkl'

with open(fpath, 'rb') as f:
    splits = pickle.load(f)
    train_audio_emb, val_audio_emb, test_audio_emb = splits

In [44]:
for d in splits:
    print(len(d))

9989
1109
2610


In [45]:
sample_val = train_audio_emb['0_0']
len(sample_val)

1611

In [46]:

# check size of feature vectors
for emb in splits:
    print(Counter(len(v) for v in emb.values()))

Counter({1611: 9989})
Counter({1611: 1109})
Counter({1611: 2610})


In [47]:
type(sample_val)

numpy.ndarray

In [50]:
# check for nan
for ds, emb in zip(('train', 'val', 'test'), splits):
    hasNan = any(np.isnan(np.sum(v)) for v in emb.values())
    print(f'{ds} {hasNan}')
    assert not hasNan

train False
val False
test False


In [78]:
# want to ensure the labels and the feature vectors match up
assert len(df['dia_utt']) == len(train_audio_emb), 'train label count unequal to train feature vectors'

In [80]:
# assert every key in the labels has a corresponding feature key
feature_keys = set(train_audio_emb)
assert all(k in feature_keys for k in df['dia_utt'])

__Must convert the emotion labels to numeric. Use ordinal encoding__


In [145]:
from sklearn.preprocessing import OrdinalEncoder

labels = np.asarray(train_df['Emotion']).reshape(-1, 1)
encoder = OrdinalEncoder()
y_train = encoder.fit_transform(labels)

sample = 20
for a, b in zip(labels[:sample], y_train[:sample]):
    print(a[0], b[0])
    
y_train.shape

neutral 4.0
neutral 4.0
neutral 4.0
neutral 4.0
surprise 6.0
neutral 4.0
neutral 4.0
neutral 4.0
neutral 4.0
neutral 4.0
fear 2.0
neutral 4.0
surprise 6.0
neutral 4.0
surprise 6.0
sadness 5.0
surprise 6.0
fear 2.0
neutral 4.0
neutral 4.0


(9989, 1)

In [148]:
sample_val

array([0.57997844, 0.97480089, 0.15641021, ..., 0.21155205, 0.46432428,
       0.2308289 ])

In [117]:
# df['audio_features'] = df['dia_utt']
# df['audio_features'] = df['audio_features'].map(train_audio_emb)
# df['audio_features']

0       [0.579978439650127, 0.9748008896749043, 0.1564...
1       [0.28147418892256576, 0.0, 0.12441514993438094...
2       [0.5014279709337522, 0.0, 0.058416431345751405...
3       [0.1388128533473851, 0.0, 0.08666499619317054,...
4       [0.25864073256920134, 0.0, 0.20901806323479546...
                              ...                        
9984    [0.2417414004975775, 0.9093490972563685, 0.062...
9985    [0.4842651681687178, 0.0, 0.11541309428669909,...
9986    [0.5233553696735176, 0.0, 0.2133521777759187, ...
9987    [0.8594383850654352, 0.0, 0.6220717276650923, ...
9988    [0.9869913859161842, 0.6145796926996386, 0.182...
Name: audio_features, Length: 9989, dtype: object

In [146]:
X_train = np.array([train_audio_emb[x] for x in train_df['dia_utt']])
X_train.shape

(9989, 1611)

In [125]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [130]:
add_dia_utt(test_df)
test_df['dia_utt']

0          0_0
1          0_1
2          0_2
3          1_0
4          1_1
         ...  
2605    279_11
2606    279_12
2607    279_13
2608    279_14
2609    279_15
Name: dia_utt, Length: 2610, dtype: object

In [131]:
X_test = np.array([test_audio_emb[x] for x in test_df['dia_utt']])
X_test.shape

(2610, 1611)

In [138]:
pred_test = clf.predict(X_test)
pred_test

array([0., 4., 0., ..., 0., 3., 4.])

In [137]:
test_labels = np.asarray(test_df['Emotion']).reshape(-1, 1)
y_test = encoder.fit_transform(test_labels) 
y_test

array([[6.],
       [0.],
       [4.],
       ...,
       [4.],
       [4.],
       [4.]])

In [139]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_test, normalize=True)

0.2421455938697318

__Quick Dim Reduction for GCP__

In [195]:
from sklearn import cluster

train_feats = np.array([train_audio_emb[x] for x in train_df['dia_utt']])
test_feats = np.array([test_audio_emb[x] for x in test_df['dia_utt']])

agglo = cluster.FeatureAgglomeration(n_clusters=400) # rougly 1/4 orginal count
agglo.fit(train_feats)

X_train_reduced = agglo.transform(train_feats)
print(X_train_reduced.shape)

X_test_reduced = agglo.transform(test_feats)
print(X_test_reduced.shape)

(9989, 400)
(2610, 400)


__Export CSV__

In [205]:
# combining train features and labels into 1 CSV for auto ML testing
temp1 = pd.DataFrame(X_train_reduced)
temp2 = train_df[['Emotion']]

train_out = pd.concat([temp1, temp2], axis=1, ignore_index=True)

train_out.rename(columns={400:'target'}, inplace=True)
train_out.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,target
0,0.866356,0.614324,0.185344,0.111245,0.648110,0.578526,0.310698,0.152067,0.674275,0.442923,...,0.968448,0.224236,0.653971,0.717273,0.627245,0.388492,0.985613,0.311949,0.780483,neutral
1,0.243612,0.505836,0.535198,0.161876,0.447502,0.226764,0.244084,0.231661,0.239147,0.658598,...,0.345272,0.465961,0.338253,0.757522,0.324574,0.493105,0.331571,0.635806,0.263701,neutral
2,0.492822,0.384429,0.047081,0.670914,0.622658,0.485583,0.188774,0.921643,0.487226,0.546144,...,0.313540,0.656917,0.473370,0.292813,0.255299,0.369913,0.473489,0.878479,0.473124,neutral
3,0.357869,0.456488,0.085393,0.099274,0.712595,0.175410,0.180970,0.811440,0.229033,0.573145,...,0.567901,0.152608,0.997614,0.299942,0.476902,0.264865,0.654398,0.578228,0.426295,neutral
4,0.045266,0.383779,0.038214,0.042278,0.714882,0.040906,0.181542,0.898459,0.046304,0.432964,...,0.125052,0.085763,0.160444,0.933145,0.186585,0.548695,0.109561,0.061698,0.456410,surprise


In [206]:
train_out.to_csv('data/train_out.csv', index=False)

In [207]:
# combine test features / labels into 1 CSV for auto ML testing
temp1 = pd.DataFrame(X_test_reduced)
temp2 = test_df[['Emotion']]

test_out = pd.concat([temp1, temp2], axis=1, ignore_index=True)

test_out.rename(columns={400: 'target'}, inplace=True)
test_out.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,target
0,0.366837,0.489585,0.304097,0.387441,0.564342,0.057354,0.248615,0.670358,0.309868,0.625212,...,0.908848,0.188460,0.348473,0.798020,0.343952,0.579691,0.701661,0.083819,0.194004,surprise
1,0.202844,0.734532,0.427303,0.020722,0.716076,0.208896,0.501804,0.122180,0.080992,0.489508,...,0.010371,0.501315,0.102205,0.718053,0.784799,0.308052,0.809099,0.254798,0.725492,anger
2,0.795905,0.333515,0.255933,0.308112,0.686258,0.650555,0.153213,0.869954,0.791233,0.598061,...,0.659105,0.675327,0.633307,0.556018,0.419217,0.382399,0.633581,0.496429,0.017109,neutral
3,0.807803,0.386059,0.351130,0.491587,0.785491,0.948802,0.138051,0.820162,0.435799,0.699574,...,0.956317,0.672169,0.715221,0.376253,0.933253,0.361340,0.715530,0.843297,0.654712,neutral
4,0.618324,0.400439,0.323686,0.325977,0.816765,0.812821,0.123878,0.895026,0.658316,0.569047,...,0.580578,0.430384,0.565957,0.884793,0.710847,0.537625,0.551684,0.769204,0.130550,joy


In [208]:
test_out.to_csv('data/test_out.csv', index=False)